# EDA of Teranet data with added columns
This notebook presents Exploratory Data Analysis (EDA) for Teranet records that were:
* cleaned and filtered for duplicates
    * `consideration_amt` < $30 reset to NaN (Not a Number, missing values)
    * records matching on all columns have been removed (83'798 records)
    * records matching on all columns excluding `pin` have been removed (729'182 records)
    * **813'138 duplicate entries** removed in total from original Teranet dataset 
    * 8'226'103 unique records remain after duplicates have been removed
    * see notebook `data_cleaning/Teranet_data_cleaning.ipynb` for details
* filtered to include only records from GTHA 
    * filtering performed via a spatial join
    * `xy` coordinates of Teranet records joined (how='inner', op='within') with DA geometry for GTHA 
    * DA geometry provided by York Municipal Government (accessed via Esri Open Data portal)
    * 6,062,853 records have `xy` coordinates within GTHA boundary
    * see notebook `data_cleaning/Teranet_GTHA_DA_spatial_join.ipynb` for details
* filtered to exclude records with missing (or under $30) `consideration_amt`
    * `consideration_amt` < $30 was reset to NaN during data cleaning
    * records with `consideration_amt` == NaN were removed before adding new columns
    * 4,637,584 records have non-NaN `consideration_amt`
    * see notebook `feature_extraction/Teranet_feature_extraction.ipynb` for details
* new columns added (available on transaction level):
    * `da_id`, `da_city`, `da_median_tot_inc`: were added during the spatial join with DA data
    * `xy`: `x` and `y` concatenated together (used for grouping by coordinate pairs)
    * `pin/xy_total_sales`: total records for this `pin`/`xy`
    * `pin/xy_prev_sales`: previous records from this `pin`/`xy` (not counting current transaction)
    * `pin/xy_price_cum_sum`: cumulative price of all records to date from this `pin`/`xy`
    * `pin/xy_price_pct_change`: price percentage change compared to previous record from this `pin`/`xy`
    * `price_da_pct_change`: price percentage change compared to previous record from this DA (by `da_id`)
    * `pin/xy_years_since_last_sale`: years since last sale from this `pin`/`xy`
    * `da_days_since_last_sale`, `da_years_since_last_sale`: days or years since last sale from this DA (by `da_id`)
    * `pin/xy_sale_next_1y/3y`, whether this `pin` or `xy` will have another sale in the future
    * `sale_next_6m/1y/3y`: "looks into the future" to see whether there is another transaction from this `pin`/`xy` within the given time horizon (6 months, 1 year, 3 years)
    * see notebook `Teranet_feature_extraction.ipynb` for details
    * data types are described below

## Preparations
### Import dependencies

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import contextily as ctx
import google_streetview.api
import time
import os
import sys

In [2]:
os.chdir('Documents/repos/geo')
sys.path.append('src')
os.listdir()

['.git',
 '.gitattributes',
 '.gitignore',
 '.idea',
 '.ipynb_checkpoints',
 'data',
 'img',
 'notebooks',
 'presentations',
 'README.md',
 'src',
 '__pycache__']

### Import user-defined functions

In [3]:
from plot_utils import plot_subset
from plot_utils import map_subset
from plot_utils import get_plot_title

In [4]:
# Set some parameters to apply to all plots. These can be overridden
# in each plot if desired
import matplotlib
# Plot size to 14" x 7"
# matplotlib.rc('figure', figsize = (14, 7))
# Font size to 14
matplotlib.rc('font', size = 14)
# Do not display top and right frame lines
matplotlib.rc('axes.spines', top = False, right = False)
# Remove grid lines
matplotlib.rc('axes', grid = False)
# Set background color to white
matplotlib.rc('axes', facecolor = 'white')

## Load Teranet records with added columns

In [ ]:
# column `pin` will be converted to dtype=category
# after records with NaN `consideration_amt` will be dropped
dtypes = {
    'decade': 'int',
    'year': 'int',
    'lro_num': 'category',
    'pin': 'category',
    'postal_code': 'category',
    'street_designation': 'category',
    'street_direction': 'category',
    'municipality': 'category',
    'da_id': 'category',
    'da_city': 'category',
}
t = time.time()
teranet_path = 'data/HHSaleHistory_cleaned_v0.9_GTHA_DA_with_cols_v0.9.csv'
df = pd.read_csv(teranet_path,
                 dtype=dtypes,
                 parse_dates=['registration_date'])
df = df.sort_values('registration_date')
elapsed = time.time() - t
print("----- DataFrame with Teranet records loaded"
      "\nin {0:.2f} seconds".format(elapsed) + 
      "\nwith {0:,} rows\nand {1:,} columns"
      .format(df.shape[0], df.shape[1]) + 
      "\n-- Column names:\n", df.columns)

KeyboardInterrupt: 

In [ ]:
df.head()

In [ ]:
df.info(null_counts=True)

## Create a GeoDataFrame from Teranet records
`x` and `y` coordinates are combined to form Point objects, CRS information for WGS84 is added.

In [ ]:
t = time.time()
# combine values in columns 'x' and 'y' into a Point geometry object
geometry = [Point(xy) for xy in zip(df['x'], df['y'])]

# generate a new GeoDataFrame by adding point geometry to data frame 'teranet_sales_data'
gdf = gpd.GeoDataFrame(df, geometry=geometry)
elapsed = time.time() - t
print("Geometry generated from 'x' and 'y' pairs, GeoDataFrame created!"
      "\ntook {0:.2f} seconds.".format(elapsed))
# add CRS for WGS84 (lat-long) to GeoDataFrame 'sales_data_geo'
gdf.crs = {'proj': 'latlong', 
           'ellps': 'WGS84', 
           'datum': 'WGS84', 
           'no_defs': True}
print("CRS dictionary for WGS84 added to geoDataFrame with Teranet records!")

## Load DA geometry for GTHA
Provided by York Municipal Government, accessed via Esri Open Data portal.

In [ ]:
api_url = 'https://opendata.arcgis.com/datasets/9d262f8a576842fbb2afbc8c51a64178_1.geojson'
da_income_gdf = gpd.read_file(api_url)
print("GeoDataFrame 'da_income_gdf' with profile of income by dissimination areas created!")
da_income_gdf.plot();

## Records per `pin` vs records per `xy` pair
There are significantly more records with unique `pin` compared to unique `xy` coordinate pairs, which appears to be reasonable (some `xy` coordinate pairs correspond to records with different`pin`s).

In [ ]:
df['pin'].describe()

In [ ]:
df['xy'].describe()

In [ ]:
# get counts of Teranet records per `pin` and `xy`
pin_counts = df.groupby('pin')['pin_total_sales'].max()
xy_counts = df.groupby('xy')['xy_total_sales'].max()
# print top 10 pins and xy by count of Teranet records
print("Top 10 pins by count of Teranet records\n")
print(pin_counts.sort_values(ascending=False).head(10))
print("\n\nTop 10 xy pairs by count of Teranet records\n")
print(xy_counts.sort_values(ascending=False).head(10))

### Histogram of records per `pin` and `xy`, for pins and xy with under 10 Teranet records

In [ ]:
# histograms for `pin` and `xy` with <=10 records
max_records = 10
f, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 6))
plt.suptitle("Histogram of Teranet records per `pin` or `xy`")
# plot histogram of records per pin
pin_counts[pin_counts <= max_records].hist(ax=axes[0])
axes[0].text(2, 700000, "Mean: {0:,.2f}"
             .format(pin_counts[pin_counts <= max_records].mean()), 
             ha='left')
axes[0].text(3, 600000, "Median: {0:,.2f}"
             .format(pin_counts[pin_counts <= max_records].median()), 
             ha='left')
axes[0].get_yaxis()\
    .set_major_formatter(plt.FuncFormatter(lambda x, 
                                    loc: "{:,}".format(int(x))))
axes[0].set_title("for pins with <=10 records")
axes[0].set_ylabel("# of pins")
axes[0].set_xlabel("# of Teranet records")
axes[0].grid(linestyle=':')

# plot histogram of records per pin
xy_counts[xy_counts <= max_records].hist(ax=axes[1])
axes[1].text(2, 450000, "Mean: {0:,.2f}"
             .format(xy_counts[xy_counts <= max_records].mean()), ha='left')
axes[1].text(3, 350000, "Median: {0:,.2f}"
             .format(xy_counts[xy_counts <= max_records].median()), ha='left')
axes[1].get_yaxis()\
    .set_major_formatter(plt.FuncFormatter(lambda x, 
                                    loc: "{:,}".format(int(x))))
axes[1].set_title("\nfor xy pairs with <=10 records")
axes[1].set_ylabel("# of xy coordinate pairs")
axes[1].set_xlabel("# of Teranet records")
axes[1].grid(linestyle=':')
f.tight_layout()

plt.show()

# histograms for all `pin` and `xy`
f, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 6))
plt.suptitle("Histogram of Teranet records per `pin` or `xy`")
# plot histogram of records per pin
pin_counts.hist(ax=axes[0], bins=100)
axes[0].text(50, 10000, "Mean: {0:,.2f}"
             .format(pin_counts.mean()), ha='left')
axes[0].text(75, 1000, "Median: {0:,.2f}"
             .format(pin_counts.median()), ha='left')
axes[0].set_yscale('log')
axes[0].get_yaxis()\
    .set_major_formatter(plt.FuncFormatter(lambda x, 
                                    loc: "{:,}".format(int(x))))
axes[0].set_title("for all pins (log scale)")
axes[0].set_ylabel("# of pins (log scale)")
axes[0].set_xlabel("# of Teranet records")
axes[0].grid(linestyle=':')

# plot histogram of records per pin
xy_counts.hist(ax=axes[1], bins=100)
axes[1].text(1000, 10000, "Mean: {0:,.2f}"
             .format(xy_counts.mean()), ha='left')
axes[1].text(1500, 1000, "Median: {0:,.2f}"
             .format(xy_counts.median()), ha='left')
axes[1].set_yscale('log')
axes[1].get_yaxis()\
    .set_major_formatter(plt.FuncFormatter(lambda x, 
                                    loc: "{:,}".format(int(x))))
axes[1].set_title("\nfor xy pairs (log scale)")
axes[1].set_ylabel("# of xy coordinate pairs (log scale)")
axes[1].set_xlabel("# of Teranet records")
axes[1].grid(linestyle=':')
f.tight_layout()

plt.show()

## Mapping `pin` and `xy` with larger number of records
### Map pins with a larger number of records

In [ ]:
# set index of GeoDataFrame with Teranet records to `pin`
gdf = gdf.set_index('pin')
print("Index of the GeoDataFrame with Teranet records set to `pin`")

#### > 10 Teranet records per `pin`
Only 465 pins (0.02% of the total) have more than 10 Teranet records.

In [ ]:
min_records = 10 # min Teranet records, to filter pins
# select subset of pins that have >min_records records
focus_ids = pin_counts[pin_counts > min_records].index
p_title = get_plot_title(pin_counts, min_records, 'pin')
map_subset(gdf, focus_ids, color_col='pin_total_sales',
           plot_title=p_title)

#### > 100 Teranet records per `pin`
Only 30 pins (0.001% of the total) have more than 100 Teranet records.

In [ ]:
min_records = 100 # min Teranet records, to filter pins
# select subset of pins that have >min_records records
focus_ids = pin_counts[pin_counts > min_records].index
p_title = get_plot_title(pin_counts, min_records, 'pin')
map_subset(gdf, focus_ids, color_col='pin_total_sales',
           plot_title=p_title, plot_alpha=1)

### Map `xy`s with a larger number of records

In [ ]:
# set index of GeoDataFrame with Teranet records to `pin`
gdf = gdf.reset_index()
gdf = gdf.set_index('xy')
print("Index of the GeoDataFrame with Teranet records reset to `xy`")
# coordinates of downtown Toronto
downtown_center = (-8840000, 5415000) 
zoom_radius = 10000 # zoom radius of 10km

#### > 10 Teranet records per `xy` pair
Only 5,350 `xy` pairs (0.38% of the total) have more than 10 Teranet records.  
However, this number is over 10 times the number of pins with >10 records.

In [ ]:
min_records = 10 # min Teranet records, to filter pins
# select subset of pins that have >min_records records
focus_ids = xy_counts[xy_counts > min_records].index
p_title = get_plot_title(xy_counts, min_records, 'xy pair')
map_subset(gdf, focus_ids, color_col='xy_total_sales',
           plot_title=p_title, plot_alpha=0.005)

#### > 100 Teranet records per `xy` pair
Only 2,985 `xy` pairs (0.21% of the total) have more than 100 Teranet records.  
However, this number is almost 100 times the number of pins with >100 records.

In [ ]:
min_records = 100 # min Teranet records, to filter pins
# select subset of pins that have >min_records records
focus_ids = xy_counts[xy_counts > min_records].index
p_title = get_plot_title(xy_counts, min_records, 'xy pair')
map_subset(gdf, focus_ids, color_col='xy_total_sales',
           plot_title=p_title, plot_alpha=0.005)

#### > 1'000 Teranet records per `xy` pair
 `xy` pairs (% of the total) have more than 1'000 Teranet records.  

In [ ]:
min_records = 1000 # min Teranet records, to filter pins
# select subset of pins that have >min_records records
focus_ids = xy_counts[xy_counts > min_records].index
p_title = get_plot_title(xy_counts, min_records, 'xy pair')
map_subset(gdf, focus_ids, color_col='xy_total_sales',
           plot_title=p_title, plot_alpha=0.005)

#### > 3'000 Teranet records per `xy` pair
 `xy` pairs (% of the total) have more than 3'000 Teranet records.  

In [ ]:
min_records = 3000 # min Teranet records, to filter pins
# select subset of pins that have >min_records records
focus_ids = xy_counts[xy_counts > min_records].index
p_title = get_plot_title(xy_counts, min_records, 'xy pair')
map_subset(gdf, focus_ids, color_col='xy_total_sales',
           plot_title=p_title, plot_alpha=1)

##### Zoom to downtown Toronto

In [ ]:
map_subset(gdf, focus_ids, color_col='xy_total_sales',
           plot_title=p_title, plot_alpha=1,
           zoom_center=downtown_center, zoom_radius=zoom_radius)

## Rolling sum and count of transactions
By `pin` or `xy`

In [ ]:
top_pins = pin_counts.sort_values(ascending=False).head(10)
top_pins

In [ ]:
focus_id = top_pins.index[0]

y1_label = "Rolling sum of transactions, $"
y2_label = "Consideration amount of each transaction, $"
x_label = "Rolling count of transactions"
plotted_subset, add, x, y = \
    plot_subset(df, focus_id, 'pin', 
                'registration_date', 'pin_price_cum_sum',
                y_col2='consideration_amt',
                p_x_label=x_label, 
                p_y1_label=y1_label, p_y2_label=y2_label,
                return_subset=True)

### Check the pin using Google Street View API

In [45]:
# local path -- linked to a private folder
key_path = '../credentials/google_maps_api.txt'
with open(key_path) as f:
    api_key = f.readlines()[0]
# Define parameters for street view api
params = [{
	'size': '600x300', # max 640x640 pixels
	'location': '46.414382,10.013988',
	'heading': '151.78',
	'pitch': '-0.76',
	'key': api_key
}]

## Distribution of cumulative sum by `pin`/`xy`

### Map pins/xy with higher cumulative sums

## Distributions of days since last sale
By `pin` or `xy`

In [19]:
df.columns

Index(['registration_date', 'decade', 'year', 'lro_num', 'pin',
       'consideration_amt', 'postal_code', 'unitno', 'street_name',
       'street_designation', 'street_direction', 'municipality',
       'street_number', 'x', 'y', 'da_id', 'da_city', 'da_median_tot_inc',
       'xy', 'pin_total_sales', 'xy_total_sales', 'pin_prev_sales',
       'xy_prev_sales', 'pin_price_cum_sum', 'xy_price_cum_sum',
       'pin_price_pct_change', 'xy_price_pct_change', 'price_da_pct_change',
       'pin_years_since_last_sale', 'xy_years_since_last_sale',
       'da_days_since_last_sale', 'da_years_since_last_sale',
       'pin_sale_next_6m', 'pin_sale_next_1y', 'pin_sale_next_3y',
       'xy_sale_next_6m', 'xy_sale_next_1y', 'xy_sale_next_3y', 'geometry'],
      dtype='object')

In [91]:
df['pin_days_since_last_sale'].hist(bins=200)
plt.title("Days since last sale, by `pin`")
plt.xlabel('')

KeyError: 'pin_days_since_last_sale'

In [ ]:
mask = df['days_since_last_sale_xy'] < 50
df.loc[mask, 'days_since_last_sale_xy'].hist(bins=50)

## Years since last sale per `pin` or `xy`

In [ ]:
se = df['pin_years_since_last_sale']
se[se < 40].hist(bins=100)

In [ ]:
se = df['xy_years_since_last_sale']
se[se < 40].hist(bins=100)

## Rolling sum vs rolling count or vs time

In [ ]:
# plot results
focus_id = df['id'].unique()[0]
plot_subset = df[df['id'] == focus_id]
plot_subset.plot(x='99_rolling_count', y='99_rolling_sum')

plt.suptitle("Rolling sum of purchases in department 99")
plt.title("for customer id: {0}".format(focus_id))
plt.ylabel("Rolling sum of purchases, $")
plt.xlabel("Rolling count of purchases by this customer in dept 99")
plt.grid(linestyle=':')
plt.show()

In [ ]:
# plot results
focus_id = df['id'].unique()[0]
plot_subset = df[df['id'] == focus_id]
plot_subset.plot(x='date', y='99_rolling_sum')

plt.suptitle("Rolling sum of purchases in department 99")
plt.title("for customer id: {0}".format(focus_id))
plt.ylabel("Rolling sum of purchases, $")
plt.grid(linestyle=':')
plt.show()